### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

By providing a company name and their primary website.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [17]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
cimb = Website("https://www.cimb.com.my")
cimb.links

['/en/personal.html',
 'https://www.cimbpreferred.com.my/en/home.html',
 'https://www.cimbprivatebanking.com/en/home.html',
 '/en/business/home.html',
 'https://www.cimb.com/en/wholesalebanking/home.html',
 'https://www.cimb.com/en/home.html',
 'https://www.cimbislamic.com/en/home.html',
 'https://www.cimb.com/en/sustainability/our-journey.html',
 '/content/cimbfoundation/en/home.html',
 '/en/personal/home.html',
 '/en/personal/home.html',
 '/bm/personal/home.html',
 '/en/personal.html',
 'https://www.cimbniaga.co.id/id/personal.html',
 'https://www.cimb.com.sg/en/personal/home.html',
 'https://www.cimbthai.com/en/personal/home.html',
 'https://www.cimbbank.com.kh/en/personal/home.html',
 'https://www.cimbbank.com.vn/vi/personal/home.html',
 'https://www.cimbbank.com.ph/en/home.html',
 'https://www.cimbclicks.com.my/clicks/#/',
 'https://www.cimb.bizchannel.com.my',
 'https://www.corpcardsonline.cimb.com/cimb-business-portal/login',
 'https://www.custodyservices.cimbbank.com.my/imc/app

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
Use "one shot prompting" 

This is an excellent use case for an LLM, because it requires nuanced understanding. 
Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!


In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [30]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links and broken links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [31]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.cimb.com.my - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links and /cimbfoundation/en/home.html.
Links (some might be relative links):
/en/personal.html
https://www.cimbpreferred.com.my/en/home.html
https://www.cimbprivatebanking.com/en/home.html
/en/business/home.html
https://www.cimb.com/en/wholesalebanking/home.html
https://www.cimb.com/en/home.html
https://www.cimbislamic.com/en/home.html
https://www.cimb.com/en/sustainability/our-journey.html
/content/cimbfoundation/en/home.html
/en/personal/home.html
/en/personal/home.html
/bm/personal/home.html
/en/personal.html
https://www.cimbniaga.co.id/id/personal.html
https://www.cimb.com.sg/en/personal/home.html
https://www.cimbthai.com/en/personal/home.html
https://www.cimbbank.com.kh/en/personal/home.html
https://www.cimbbank.com.vn/vi/personal/h

In [32]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
## Ollama version - Unable to provide info
# OLLAMA_API = "http://localhost:11434/api/chat"
# HEADERS = {"Content-Type": "application/json"}
# MODEL_llama = "llama3.2"

# ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# def get_links(url):
#     website = Website(url)
#     response = ollama_via_openai.chat.completions.create(
#         model=MODEL_llama,
#         messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": get_links_user_prompt(website)}
#       ],
#         response_format={"type": "json_object"}
#     )
#     result = response.choices[0].message.content
#     return json.loads(result)

In [33]:
cimb = Website("https://www.cimb.com.my")
cimb.links

['/en/personal.html',
 'https://www.cimbpreferred.com.my/en/home.html',
 'https://www.cimbprivatebanking.com/en/home.html',
 '/en/business/home.html',
 'https://www.cimb.com/en/wholesalebanking/home.html',
 'https://www.cimb.com/en/home.html',
 'https://www.cimbislamic.com/en/home.html',
 'https://www.cimb.com/en/sustainability/our-journey.html',
 '/content/cimbfoundation/en/home.html',
 '/en/personal/home.html',
 '/en/personal/home.html',
 '/bm/personal/home.html',
 '/en/personal.html',
 'https://www.cimbniaga.co.id/id/personal.html',
 'https://www.cimb.com.sg/en/personal/home.html',
 'https://www.cimbthai.com/en/personal/home.html',
 'https://www.cimbbank.com.kh/en/personal/home.html',
 'https://www.cimbbank.com.vn/vi/personal/home.html',
 'https://www.cimbbank.com.ph/en/home.html',
 'https://www.cimbclicks.com.my/clicks/#/',
 'https://www.cimb.bizchannel.com.my',
 'https://www.corpcardsonline.cimb.com/cimb-business-portal/login',
 'https://www.custodyservices.cimbbank.com.my/imc/app

In [34]:
get_links("https://www.cimb.com.my")

{'links': [{'type': 'about page', 'url': 'https://www.cimb.com/en/home.html'},
  {'type': 'about page',
   'url': 'https://www.cimb.com/en/sustainability/our-journey.html'},
  {'type': 'careers page',
   'url': 'https://www.cimb.com/en/business/home.html'},
  {'type': 'careers page', 'url': 'https://apply.cimb.com/my/careers'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [35]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [36]:
# print(get_all_details("https://huggingface.co"))
print(get_all_details("https://www.cimb.com.my"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cimb.com/en/home.html'}, {'type': 'business page', 'url': 'https://www.cimb.com/en/business/home.html'}, {'type': 'careers page', 'url': 'https://www.cimb.com/en/careers.html'}, {'type': 'wholesale banking page', 'url': 'https://www.cimb.com/en/wholesalebanking/home.html'}, {'type': 'sustainability page', 'url': 'https://www.cimb.com/en/sustainability/our-journey.html'}, {'type': 'personal banking page', 'url': 'https://www.cimbpreferred.com.my/en/home.html'}, {'type': 'private banking page', 'url': 'https://www.cimbprivatebanking.com/en/home.html'}, {'type': 'company page', 'url': 'https://www.cimb.com/en/home.html'}, {'type': 'investments page', 'url': 'https://www.cimb.com.my/en/personal/wealth-management/investments/e-gold-investment-account-egia.html'}]}
Landing page:
Webpage Title:
Personal Banking | Savings, Credit Cards and Loans | CIMB Malaysia
Webpage Contents:
Close
Search
Login
CIMB Clicks
BizChannel@CIMB
Co

In [37]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [38]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [39]:
get_brochure_user_prompt("CIMB", "https://www.cimb.com.my")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cimb.com/en/home.html'}, {'type': 'about page', 'url': 'https://www.cimb.com/en/sustainability/our-journey.html'}, {'type': 'careers page', 'url': 'https://www.cimb.com.my/en/personal/help-support/contact-us.html'}, {'type': 'careers page', 'url': 'https://www.cimb.com.my/en/personal/important-notices.html'}, {'type': 'company page', 'url': 'https://www.cimb.com.my/en/business/solutions-products/solutions/sme-banking.html'}, {'type': 'promotions page', 'url': 'https://www.cimb.com.my/en/personal/promotions/latest-promotions.html'}, {'type': 'promotions page', 'url': 'https://www.cimb.com.my/en/personal/general-terms-conditions.html'}, {'type': 'company social media', 'url': 'https://www.facebook.com/CIMBMalaysia/'}, {'type': 'company social media', 'url': 'https://www.instagram.com/cimbmalaysia/?hl=ms'}, {'type': 'company social media', 'url': 'https://www.linkedin.com/company/cimb'}]}


'You are looking at a company called: CIMB\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nPersonal Banking | Savings, Credit Cards and Loans | CIMB Malaysia\nWebpage Contents:\nClose\nSearch\nLogin\nCIMB Clicks\nBizChannel@CIMB\nCorporate Cards Online\nOther Services\nCustody Services\nDay To Day Banking\nIslamic Banking\nWealth Management\nDeals & Promotions\nDigital Services\nHelp & Support\nAccounts\nCards\nFinancing\nRemittance\nSustainability at CIMB\nCIMB@Work\nMore Services\nIslamic Banking Overview\nIslamic Wealth Management\nMore Services\nInvestments\nInsurance/Takaful\nLatest Promotions\nCIMB Deals\nCIMB OCTO App\nCIMB Clicks\nCIMB Partner Pay Solution\nApply for Products\nDuitNow QR\nPersonalised For You\nCustomer Help Centre\nLocate Us\nRates & Charges\nCalculators\nSecurity & Fraud\nSavings Account\nCurrent Account\nFixed Deposit Account\nMudara

In [40]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [41]:
create_brochure("CIMB", "https://www.cimb.com.my")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cimb.com/en/home.html'}, {'type': 'wholesale banking page', 'url': 'https://www.cimb.com/en/wholesalebanking/home.html'}, {'type': 'preferred banking page', 'url': 'https://www.cimbpreferred.com.my/en/home.html'}, {'type': 'private banking page', 'url': 'https://www.cimbprivatebanking.com/en/home.html'}, {'type': 'Islamic banking page', 'url': 'https://www.cimbislamic.com/en/home.html'}, {'type': 'sustainability page', 'url': 'https://www.cimb.com/en/sustainability/our-journey.html'}, {'type': 'careers page', 'url': 'https://www.cimb.com.my/en/personal/help-support/contact-us.html'}, {'type': 'jobs page', 'url': 'https://www.cimb.com.my/en/personal/help-support/contact-us.html'}]}


# CIMB Company Brochure

![CIMB Logo](insert_logo_url_here) 

## About CIMB

CIMB is one of the leading financial services providers in Southeast Asia, dedicated to delivering exceptional banking solutions across multiple markets including Malaysia, Indonesia, Singapore, Thailand, Cambodia, Vietnam, and the Philippines. We pride ourselves on our comprehensive range of products tailored to meet the diverse needs of our customers, from personal banking and credit cards to financing and wealth management services.

---

## Our Services

- **Personal Banking:** Offering a variety of accounts, credit cards, personal and property financing to suit individual needs.
- **Islamic Banking:** Adhering to Shariah principles, we provide a robust suite of Islamic banking and wealth management services.
- **Business & Corporate Solutions:** Comprehensive banking services for businesses, including financing and cash management solutions.
- **Digital Services:** Our innovative digital platforms such as CIMB Clicks and CIMB OCTO App make banking accessible and convenient.

---

## Customer-Centric Approach

At CIMB, we believe in putting our customers first. Our initiatives include:
- **Promotions and Rewards:** Engage with exciting campaigns that offer customers opportunities to win prizes and bonuses.
- **Personalized Solutions:** We tailor our financial products to meet the unique needs of our diverse clientele.

### Current Promotions:
- **Riuh Durian Runtuh Campaign:** Open a DURIAN-i Account and stand to win prizes and bonuses.
- **Cash Plus Personal Loan:** Instant approvals with no collateral required for loans up to RM100,000.

---

## Our Commitment to Sustainability

CIMB is committed to sustainable banking. Our initiatives aim to balance economic growth with environmental and social awareness, ensuring a positive impact on our communities and the planet.

---

## Company Culture

CIMB fosters a dynamic work environment that promotes inclusivity, innovation, and continuous learning. Our culture emphasizes:
- **Collaboration:** Encouraging teamwork across departments to achieve shared goals.
- **Innovation:** Embracing technology and new ideas to enhance customer experiences.
- **Diversity & Inclusion:** Valuing diverse perspectives and backgrounds, creating an enriching workplace for all.

---

## Careers at CIMB

We are always on the lookout for talented individuals to join our team. Opportunities abound in various sectors, including banking, technology, finance, and customer service. At CIMB, you can expect:
- **Professional Development:** Access to training and career advancement opportunities.
- **Supportive Environment:** A culture that supports your personal and professional growth.

### Join Us!
Explore career opportunities at [CIMB Careers](insert_careers_page_url_here).

---

## Connect with Us

For more information, visit our website at [CIMB Malaysia](insert_website_url_here) or follow us on our official social media channels.

---

Thank you for considering CIMB as your banking partner, employer, and investor! Together, we can achieve great things.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("CIMB", "https://www.cimb.com.my")